<a href="https://colab.research.google.com/github/jhk990602/datapractice/blob/main/deep_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#케라스 API를 사용해서 패션 MNIST 데이터셋 불러오기
from tensorflow import keras
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [3]:
#픽셀값을 0~255 범위에서 0~1 사이로 변환, 28*28 크기의 2차원 배열을 784 크기의 1차원 배열로 + train_test_split()함수로 훈련세트와 검증세트로 나눔
from sklearn.model_selection import train_test_split
train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1,28*28)
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state= 42)

In [6]:
#입력층과 출력층 사이에 은닉층이라는 밀칩층을 추가, 은닉층에는 주황색 원으로 활성화 함수가 표시,
#시그모이드 활성화 함수를 사용한 은닉층과 소프트맥스 함수를 사용한 출력층을 케라스의 Dense 클래스로 + 케라스에서 신경망의 첫번째층은 input_shape매개변수로 입력의 크기를 꼭지정해야함
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape =(784,)) #dense1이 은닉층이고 100개의 뉴런을 가진 밀집층, 적어도 은닉층의 뉴런개수는 출력층의 뉴런보다는 많게 만들어야함
dense2 = keras.layers.Dense(10, activation = 'softmax') #dense2는 출력층, 10개의 클래스를 분류하므로 10갸의 뉴런을 두었음

In [8]:
#dense1과 dense2 객체를 Sequentail 클래스에 추가하여 심층신경망을 만들기
model = keras.Sequential([dense1, dense2]) #Sequential 클래스의 객체를 만들 때 여러 개의 층을 추가하려면, 출력층을 가장 마지막에 두어야함

In [9]:
model.summary() #케라스는 모델의 summary() 메서드를 호출하면 층에 대한 유용한 정보를 얻을 수 있음.

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


첫 줄에 모델의 이름, 다음 이 모델에 들어있는 층이 순서대로 나열
출력크기를 보면 (None,100) 첫번째 차원은 샘플의 개수, 아직 샘플개수가 정의되어있지 않아 None, 케라스 모델의 fit()메서드에 훈련데이터를 주입하면 한 번에 이 데이터를 모두 사용하지 않고 잘게 나누어 여러번에 걸쳐 경사 하강법단계를 수행(미니배치경사하강법), 케라스의 기본 미니배치 크기는 32개, 샘플 개수를 고정하지 않고 어떤 배치크기에도 유연하게 대응할 수 있도록 None으로 설정.
두 번째 100은 은닉층의 뉴런 개수를 100개로 놓았으니 100개의 출력, 즉 샘플마다 784개의 픽셀값이 은닉층을 통과하면서 100개의 특성으로 압축.
마지막 모델파라미터개수 => 입력픽셀 784개와 100개의 모든조합에대한 가중치, 784 * 100 + 100, 두 번째층의 출력 뉴런 개수 10개, 이 층의 모델 파라미터 개수는 100개의 은닉층 뉴런과 10개의 출력층 뉴런이 모두 연결되고 출력층의 뉴런마다 하나의 절편이 있기 때문에 총 100*10+10 총 1010의 모델 파라미터가 있음

In [10]:
#층을 추가하는 다른방법
#Sequential 클래스의 생성자 안에서 바로 Dense클래스의 객체를 만들기
model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,),
                       name = 'hidden'),
    keras.layers.Dense(10, activation='softmax', name = 'output')
], name = '패션 MNIST 모델')

In [11]:
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [12]:
#Sequential 클래스의 객체를 만들고 이 객체의 add() 메서드를 호출하여 층을 추가하는 방법
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation = 'sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation = 'softmax'))

In [13]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [15]:
#모델 훈련
model.compile(loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')
model.fit(train_scaled, train_target, epochs= 5)

Epoch 1/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.5676 - accuracy: 0.8063
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.4101 - accuracy: 0.8514
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3749 - accuracy: 0.8636
Epoch 4/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3515 - accuracy: 0.8724
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3335 - accuracy: 0.8798


추가한 층이 성능을 향상시킴

시그모이드 함수는 끝으로 갈수록 그래프가 누워있어 올바른 출력을 만ㄷ느늗 ㅔ신속하게 대응하지 못하는 단점이 있음. 특히 층이 많은 심층 신경망일수록 그 효과가 누적되어 학습을 어렵게 만듬, 개선하기위해 렐루 함수사용, 입력이 양수일 경우 마치 활성화 함수가 없는 것처럼 입력을 통과시킴, 음수일 경우 0으로 만듬  +렐루함수는 특히 이미지처리에 좋은 성능 ++keras 함수에서는 28*28 크기를 인공 신경망에 주입하기 위해 넘파이 배열의 reshape()메서드를 사용해 1차원으로 펼쳤으나, Flatten 클래스를 제공. Flatten클래스는 배치 차원을 제외하고 나머지 입력차원을 모두 일렬로 펼치는 역할만 함.그러나 층처럼 입력층과 은닉층 사이에 추가하기 때문에 층이라부름.

In [16]:
#Flatten층은 입력층 바로뒤에 추가, 그러나 Flatten 층은 학습하는 층이 아니기에 Flatten 클래스에 포함된 모델 파라미터는 0개
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [17]:
#훈련데이터를 다시 준비해서 모델 훈련
(train_input, train_target),(test_input, test_target) = keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0
#train_scaled = train_scaled.reshape(-1, 28*28) 이 생략됨 (Flatten으로인해)
train_scaled, val_scaled, train_target, val_target = train_test_split( train_scaled, train_target, test_size = 0.2, random_state = 42)

In [20]:
#모델을 컴파일하고 훈련하는 것은 동일
model.compile(loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')
model.fit(train_scaled, train_target, epochs =5)

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5307 - accuracy: 0.8128
Epoch 2/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3919 - accuracy: 0.8577
Epoch 3/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3532 - accuracy: 0.8716
Epoch 4/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3314 - accuracy: 0.8803
Epoch 5/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3163 - accuracy: 0.8871


sigmoid 함수를 사용했을때와 비교해 성능이 약간 향상

In [21]:
#검증세트에서의 성능확인
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3564 - accuracy: 0.8740


[0.3564372956752777, 0.8740000128746033]

In [22]:
#Adam 클래스의 매개변수 기본값을 사용해 패션 MNIST 모델을 훈련
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100, activation = 'relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [24]:
#compile() 메서드의 optimizer를 'adam'으로 설정하고 5번의 에포크동안 훈련
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 6s 3ms/step - loss: 0.5168 - accuracy: 0.8206
Epoch 2/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3878 - accuracy: 0.8598
Epoch 3/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3491 - accuracy: 0.8736
Epoch 4/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3242 - accuracy: 0.8824
Epoch 5/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3029 - accuracy: 0.8886


In [25]:
#검증세트 평가
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3352 - accuracy: 0.8802


[0.33516228199005127, 0.8801666498184204]